In [1]:
!pip install torch torchvision numpy tqdm

In [4]:
import torch
from torch import nn, optim
from torch.utils.data import DataLoader, Dataset
from torchvision import datasets
import numpy as np

class CustomFashionMNIST(Dataset):
    def __init__(self, root='./data', train=True, transform=None):
        self.dataset = datasets.FashionMNIST(root=root, train=train, download=True)
        self.data = self.dataset.data.float().unsqueeze(1) / 255.0
        self.labels = self.dataset.targets
        self.transform = transform

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        image, label = self.data[idx], self.labels[idx]
        if self.transform:
            image = self.transform(image)
        return image, label

class FashionModel(nn.Module):
    def __init__(self):
        super(FashionModel, self).__init__()
        self.model = nn.Sequential(
            nn.Conv2d(1, 32, 3, padding=1), nn.BatchNorm2d(32), nn.ReLU(),
            nn.MaxPool2d(2, 2),
            nn.Conv2d(32, 64, 3, padding=1), nn.BatchNorm2d(64), nn.ReLU(),
            nn.MaxPool2d(2, 2),
            nn.Conv2d(64, 128, 3, padding=1), nn.BatchNorm2d(128), nn.ReLU(),
            nn.Flatten(),
            nn.Linear(128 * 7 * 7, 256), nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(256, 10)
        )

    def forward(self, x):
        return self.model(x)

class ModelTrainer:
    def __init__(self, model, device, learning_rate=0.001):
        self.model = model.to(device)
        self.device = device
        self.criterion = nn.CrossEntropyLoss()
        self.optimizer = optim.Adam(self.model.parameters(), lr=learning_rate)
        self.best_accuracy = 0.0

    def train_epoch(self, data_loader):
        self.model.train()
        running_loss = 0.0
        for images, labels in data_loader:
            images, labels = images.to(self.device), labels.to(self.device)
            self.optimizer.zero_grad()
            outputs = self.model(images)
            loss = self.criterion(outputs, labels)
            loss.backward()
            self.optimizer.step()
            running_loss += loss.item()
        return running_loss / len(data_loader)

    def evaluate(self, data_loader):
        self.model.eval()
        correct, total = 0, 0
        with torch.no_grad():
            for images, labels in data_loader:
                images, labels = images.to(self.device), labels.to(self.device)
                outputs = self.model(images)
                _, predicted = torch.max(outputs, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()
        return 100 * correct / total

    def save_best_model(self, epoch, accuracy, filename='CNN_Model.pt'):
        if accuracy > self.best_accuracy:
            self.best_accuracy = accuracy
            torch.save({
                'epoch': epoch,
                'model_state_dict': self.model.state_dict(),
                'accuracy': accuracy
            }, filename)
            print(f"New best model saved with accuracy {accuracy:.2f}% at epoch {epoch}.")

    def load_model(self, filename='CNN_Model.pt'):
        checkpoint = torch.load(filename, map_location=self.device)
        self.model.load_state_dict(checkpoint['model_state_dict'])
        self.best_accuracy = checkpoint['accuracy']
        print(f"Model loaded with accuracy {self.best_accuracy:.2f}%.")

def main():
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f"Using device: {device}")

    train_dataset = CustomFashionMNIST(train=True)
    test_dataset = CustomFashionMNIST(train=False)
    train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
    test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

    model = FashionModel()
    trainer = ModelTrainer(model, device)

    num_epochs = 108
    print("Starting training")
    for epoch in range(1, num_epochs + 1):
        train_loss = trainer.train_epoch(train_loader)
        print(f"Epoch {epoch} | Training Loss: {train_loss:.4f}")

        test_accuracy = trainer.evaluate(test_loader)
        print(f"Epoch {epoch} | Test Accuracy: {test_accuracy:.2f}%")

        trainer.save_best_model(epoch, test_accuracy)

    print(f"\nTraining complete. Best accuracy: {trainer.best_accuracy:.2f}%")

if __name__ == "__main__":
    main()


Using device: cuda
Starting training
Epoch 1 | Training Loss: 0.4207
Epoch 1 | Test Accuracy: 86.70%
New best model saved with accuracy 86.70% at epoch 1.
Epoch 2 | Training Loss: 0.2905
Epoch 2 | Test Accuracy: 89.24%
New best model saved with accuracy 89.24% at epoch 2.
Epoch 3 | Training Loss: 0.2478
Epoch 3 | Test Accuracy: 91.40%
New best model saved with accuracy 91.40% at epoch 3.
Epoch 4 | Training Loss: 0.2139
Epoch 4 | Test Accuracy: 91.20%
Epoch 5 | Training Loss: 0.1904
Epoch 5 | Test Accuracy: 92.19%
New best model saved with accuracy 92.19% at epoch 5.
Epoch 6 | Training Loss: 0.1695
Epoch 6 | Test Accuracy: 92.23%
New best model saved with accuracy 92.23% at epoch 6.
Epoch 7 | Training Loss: 0.1466
Epoch 7 | Test Accuracy: 92.16%
Epoch 8 | Training Loss: 0.1307
Epoch 8 | Test Accuracy: 92.33%
New best model saved with accuracy 92.33% at epoch 8.
Epoch 9 | Training Loss: 0.1154
Epoch 9 | Test Accuracy: 92.72%
New best model saved with accuracy 92.72% at epoch 9.
Epoch 10 